<a href="https://colab.research.google.com/github/mgranchelli/ai-projects-examples/blob/main/RunningInjuryPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Injury prediction for Competitive Runners

**Context🏃**

Running is a great form of exercise, recreation, and sport participation for adults, adolescents, and children. Whether alone or in a team environment, running, when done properly, can enhance physical fitness, coordination, sense of accomplishment, and physical and emotional development. However, running under adverse conditions or with inadequate clothing and equipment can cause a variety of injuries and physical stress.

**Content🏃**

The data set consists of a detailed training log from a Dutch high-level running team over a period of seven years (2012-2019). We included the middle and long-distance runners of the team, that is, those competing on distances between the 800 meters and the marathon. This design decision is motivated by the fact that these groups have strong endurance-based components in their training, making their training regimes comparable. The head coach of the team did not change during the years of data collection.

The data set contains samples from 74 runners, of whom 27 are women and 47 are men. At the moment of data collection, they had been in the team for an average of 3.7 years. Most athletes competed on a national level, and some also on an international level. The study was conducted according to the requirements of the Declaration of Helsinki and was approved by the ethics committee of the second author’s institution

**Acknowledgements🏃**

S. Lovdal, Ruud J. R. Den Hartigh, G. Azzopardi, "Injury Prediction in Competitive Runners with Machine Learning", International Journal of Sports Physiology and Performance, 2020, in press.

## Download data

In [ ]:
!pip install gdown -q

In [ ]:
!gdown https://drive.google.com/uc?id=10bur-F5sEANpKvKiIwi_exK2DCMe2fRh

In [ ]:
!unzip -q injury.zip

## Import modules

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import datasets, metrics

                                            Introduction
Running is one of the most popular sports in the world. [60 million](http://https://www.statista.com/topics/1743/running-and-jogging/#topicHeader__wrapper) people participated in jogging, running, or trail running in America alone in 2017. But it is reported that [50%](https://www.yalemedicine.org/conditions/running-injury#:~:text=In%20fact%2C%20at%20least%2050,People%20who%20run%2C%20love%20it.) of runners get injured every year. Dealing with injuries can be a hard and long process, which is why runners take many precautions to mitigate the chance of injuries. These precautions include investments in rollers, massages, and professional coaching to name a few. But these resources take investments that many people can not afford. With the advances in data science, many people are wondering wether machine learning will be able to reduce the barrier for injury prediction resources. In our analysis we will see how popular machine learning algorithms deal with predciting injuries using running data

## Load data

In [ ]:
df = pd.read_csv("injury/week_approach_maskedID_timeseries.csv")
np.random.seed(0)

We will do some basic data exploration to understand if the data set is dirty.

In [ ]:
missing_Values = df.isnull().sum()
missing_Values

In [ ]:
df.describe()

In [ ]:
df.head(10)

## Remove data

 The data is too high dimension for us to start data analysis. We will start off by dropping attributes based on empirical analysis. Any attributes based on how the person feels was removed from the data set. We wanted to do see if we could predict data strictly based on the quality of running using quantative data. Although 'recovery' attributes could be useful, its very hard to accurately understand how a runners body is feeling based off of survey questions.

In [ ]:
df.info()
df = df.drop(['avg training success', 'min training success', 'max training success', 'avg training success.1', 'max training success.1', 'min training success.1'], axis = 1)
df = df.drop(['avg training success.2', 'max training success.2', 'min training success.2', 'avg exertion', 'min exertion', 'max exertion'], axis = 1)
df = df.drop(['avg exertion.1', 'min exertion.1', 'max exertion.1', 'avg exertion.2', 'min exertion.2', 'max exertion.2', 'max km one day'], axis = 1)
df = df.drop(['avg recovery', 'min recovery', 'max recovery', 'avg recovery.1', 'min recovery.1', 'max recovery.1', 'avg recovery.2', 'min recovery.2', 'max recovery.2'], axis = 1)
df = df.drop(['rel total kms week 0_1', 'rel total kms week 0_2', 'rel total kms week 1_2'], axis = 1)
df.info()

We have succesfully reduced the attributes of the data set from 71 to 40. This is a good start but the data set is still to high dimensional.

## Data Exploration

In [ ]:
df['Athlete ID'].unique()

In total there are 74 athletes. Let's isolate the first athlete and see what their training looks like.


In [ ]:
def indexIndividualData(id):
  df0 = df[df['Athlete ID'] == id]
  index1 = df0.index[0]
  indexLast = df0.index[-1]
  y = indexLast - len(df0[df0['injury']==0]) - len(df0[df0['injury']==1])
  df0 = df0.rename(index = lambda x: x - y - 1 if x > indexLast - len(df0[df0['injury']==1]) else x - index1)
  df0 = df0.sort_values(by = 'Date')
  return df0
def plotIndividualData(id, column):
  df0 = indexIndividualData(id)
  plt.figure(figsize = (14,6))
  sns.lineplot(data=df0[column])

plotIndividualData(1, "total kms")

This graph does lead us to some questions. Why is there a drop in training for such a long period for this athlete? Assuming each data point is a week, over one hundred weeks of being injured does not make sense. It could make more sense if the points are actually days, but one hundred days injured is also a lot.

In [ ]:
def dateInjurySubset(id, column1, column2, column3):
  df0 = indexIndividualData(id)
  return df0[[column1,column2,column3]]

def plotIndividualDataDuoColumn(id, column1, column2, column3):
  df0 = dateInjurySubset(id,column1,column2,column3)
  plt.figure(figsize = (14,6))
  sns.lineplot(data=df0)

print(dateInjurySubset(1,"total kms", "total kms.1","total kms.2").mean())
plotIndividualDataDuoColumn(1,"total kms", "total kms.1", "total kms.2")
print(dateInjurySubset(1,"nr. sessions", "nr. sessions.1","nr. sessions.2").mean())
plotIndividualDataDuoColumn(1,"nr. sessions", "nr. sessions.1","nr. sessions.2")
print(dateInjurySubset(2,"total kms", "total kms.1","total kms.2").mean())
plotIndividualDataDuoColumn(2,"total kms", "total kms.1", "total kms.2")
print(dateInjurySubset(2,"nr. sessions", "nr. sessions.1","nr. sessions.2").mean())
plotIndividualDataDuoColumn(2,"nr. sessions", "nr. sessions.1","nr. sessions.2")

Since there does not seem to be any difference between the attributes and their ".1",".2" siblings, the attributes seem to be just noise. It could be justification to drop all the suffix attributes, as we are not sure what they contribute to the data

In [ ]:
dfQ2 = df[['Athlete ID', 'total km Z3-Z4-Z5-T1-T2', 'total km Z3-Z4-Z5-T1-T2.1', 'total km Z3-Z4-Z5-T1-T2.2', 'injury', 'nr. tough sessions (effort in Z5, T1 or T2)', 'nr. tough sessions (effort in Z5, T1 or T2).1', 'nr. tough sessions (effort in Z5, T1 or T2).2', 'total km Z5-T1-T2', 'total km Z5-T1-T2.1', 'total km Z5-T1-T2.2', 'total km Z3-4', 'total km Z3-4.1', 'total km Z3-4.2']]
dfArray = []
for i in df['Athlete ID'].unique():
  dfArray.append(indexIndividualData(i))

The 'dates' attribute is also confusing in this data set. We will try and visualize it to get a better idea of what it means.

In [ ]:
df0 = dfArray[1]
injury = df0[df0['injury'] == 1]
notInjured = df0[df0['injury'] == 0]
print("INJURED DATES ID 1")
for i in injury['Date']:
  print(i)
print("NOT INJURED DATES ID 1:\n")
for i in notInjured['Date']:
  print(i)

Why the 'dates' attribute jumps from 400 to 700 for an athlete does not make sense to us. In order to accurately predict date there should be consecutive date that can be analyzed. We could be missing something but we cant attemt to try and classify the running data anyways. The data exploration also showed us that the data set is extremely biased towards the non-injured data.

In [ ]:
plt.figure(figsize=(8, 8))
sns.countplot(x='injury', data=df0)
plt.title('Unbalanced Classes')
plt.show()

Here we can see just how biased the data set is for non-injured cases.

## Balancing dataset

In [ ]:
df1 = df0.sort_values(by = 'Athlete ID');

shuffled_df1 = df.sample(frac=1,random_state=4)

# Put all the fraud class in a separate dataset.
injury_df1 = shuffled_df1.loc[shuffled_df1['injury'] == 1]

#Randomly select 492 observations from the non-fraud (majority class)
non_injured_df1 = shuffled_df1.loc[shuffled_df1['injury'] == 0].sample(n=575)

# Concatenate both dataframes again
normalized_df = pd.concat([injury_df1, non_injured_df1])

#plot the dataset after the undersampling
plt.figure(figsize=(8, 8))
sns.countplot(x='injury', data=normalized_df)
plt.title('Balanced Classes')
plt.show()

Here we balance out the data set using sampling. This should allow us to avoid overffiting in our predictive models.

Below we will start of by showing what happens when the data set is skewed. You will see that the accruacy is very high for the unbalanced data set because the classifier will identify all the data points as non-injured. This would be uselss for the goal of this project.

In [ ]:
y = df0['injury']
X = df0.drop('injury', axis=1)
X = df0.drop('Athlete ID', axis=1)
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.3, random_state = 0)

K = []
training = []
test = []
scores = {}

for k in range(2, 21):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train, y_train)

    training_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    K.append(k)

    training.append(training_score)
    test.append(test_score)
    scores[k] = [training_score, test_score]

for keys, values in scores.items():
    print(keys, ':', values)

In [ ]:
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import RocCurveDisplay
def plot_confusion_matrix(y_true, y_pred):
    labels = unique_labels(y_true)
    columns = [f'Predicted {label}' for label in labels]
    index = [f'Actual {label}' for label in labels]
    table = pd.DataFrame(confusion_matrix(y_true, y_pred),
                         columns=columns, index=index)
    return sns.heatmap(table, annot=True, fmt='d', cmap='viridis')

In [ ]:
y_pred = clf.predict(X_test)
plot_confusion_matrix(y_test, y_pred)
svc_disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.show()

The confusion matrix above shows that all data points are identified as non-injured. We will try and use the balanced data set next.

In [ ]:
y = normalized_df['injury']
X = normalized_df.drop('injury', axis=1)
X = normalized_df.drop('Athlete ID', axis=1)
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.3, random_state = 0)

K = []
training = []
test = []
scores = {}

for k in range(2, 21):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train, y_train)

    training_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    K.append(k)

    training.append(training_score)
    test.append(test_score)
    scores[k] = [training_score, test_score]

for keys, values in scores.items():
    print(keys, ':', values)

Here we can see that as we increase the paramater k, the training accuracy goes down significantly (from about 80% to 65%) and the testing accuracy increases slighlty (from about 58% to 60%). We will use the confusion matrix to see how well the predictive model is handling the non-injured data compared to the injured data.

In [ ]:
 clf = KNeighborsClassifier(n_neighbors = 2)
 clf.fit(X_train, y_train)

 training_score = clf.score(X_train, y_train)
 test_score = clf.score(X_test, y_test)
 K.append(k)

 training.append(training_score)
 test.append(test_score)
 scores[k] = [training_score, test_score]

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
plot_confusion_matrix(y_test, y_pred)
svc_disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.show()

At the k value of 2, the classifier seems to be more accurate at predicintg the non-injured data.

In [ ]:
 clf = KNeighborsClassifier(n_neighbors = 12)
 clf.fit(X_train, y_train)

 training_score = clf.score(X_train, y_train)
 test_score = clf.score(X_test, y_test)
 K.append(k)

 training.append(training_score)
 test.append(test_score)
 scores[k] = [training_score, test_score]

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
plot_confusion_matrix(y_test, y_pred)
svc_disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.show()

With a high k value of 21, you get more acurate injury prediction, but that also means a lot more false positives for non-injure data, meaning the classifier identifies those people who are not injured as injured at a much higher rate.  

The equilibrium that we found was a k parameter value of 12, with an overall accuracy rate of 60%, 52% for non-injured data points and 68% accuracy for predicting injured data points. This is decent considering how biased the data set is but there are other binary classifiers and other balancing method that we will test to see if we can get better accuracy.

In [ ]:
#SVM classifier using undersampling
from imblearn.under_sampling import RandomUnderSampler
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
rus = RandomUnderSampler(random_state=0)
X_train, Y_train =rus.fit_resample(X_train,Y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(classifier, X_test, Y_test)
plt.show()

This code involves using a support vector machine classifier to predict whether a data point is injured or not using the undersampling technique to counter our imbalanced data set.

In [ ]:
#SVM Classifier using oversampling
from imblearn.over_sampling import SMOTE
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
sm = SMOTE(random_state = 0)
X_train, Y_train = sm.fit_resample(X_train,Y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(classifier, X_test, Y_test)
plt.show()

This code involves using a support vector machine classifier to predict whether a data point is injured or not using the oversampling technique to counter our imbalanced data set.

In [ ]:
#Bagging Classifier With Undersampling
import sklearn.ensemble
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
rus = RandomUnderSampler(random_state=0)
X_train, Y_train =rus.fit_resample(X_train,Y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
bag = sklearn.ensemble.BaggingClassifier(n_estimators = 35)
bag.fit(X_train, Y_train)
Y_pred = bag.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(bag, X_test, Y_test)
plt.show()

This code involves using a bagging classifier to predict whether a data point is injured or not using the undersampling technique to counter our imbalanced data set.

In [ ]:
#Bagging Classifier With Oversampling
import sklearn.ensemble
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
sm = SMOTE(random_state = 0)
X_train, Y_train = sm.fit_resample(X_train,Y_train)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
bag = sklearn.ensemble.BaggingClassifier(n_estimators = 30)
bag.fit(X_train, Y_train)
Y_pred = bag.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(bag, X_test, Y_test)
plt.show()

This code involves using a bagging classifier to predict whether a data point is injured or not using the oversampling technique to counter our imbalanced data set.

In [ ]:
#XGBooster model with Undersampling
from xgboost import XGBClassifier
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
rus = RandomUnderSampler(random_state=0)
X_train, Y_train =rus.fit_resample(X_train,Y_train)
boost = XGBClassifier(max_depth = 3, n_estimators = 30)
boost.fit(X_train, Y_train)
Y_pred = boost.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(boost, X_test, Y_test)
plt.show()

This code involves using the XGBooster classifier to predict whether a data point is injured or not using the undersampling technique to counter our imbalanced data set.

In [ ]:
#XGBooster model with Oversampling
from xgboost import XGBClassifier
X = df.drop('injury', axis = 1)
Y = df['injury']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y)
sm = SMOTE(random_state = 0)
X_train, Y_train = sm.fit_resample(X_train,Y_train)
boost = XGBClassifier(max_depth = 2, n_estimators = 30)
boost.fit(X_train, Y_train)
Y_pred = boost.predict(X_test)
plot_confusion_matrix(Y_test, Y_pred)
svc_disp = RocCurveDisplay.from_estimator(boost, X_test, Y_test)
plt.show()

This code involves using a bagging classifier to predict whether a data point is injured or not using the oversampling technique to counter our imbalanced data set.

Overall, the results of the models showed that the model accuracy was more dependent on whether oversampling or undersampling was used. The oversampling method was more successful at classifying data points that were actually non-injured, while the undersampling method did a much better job at accurately classifying the points that were actually injured.

                                      Conclusion
We balanced the data set using multiple different strategies, such as sampling, oversampling, and undersampling. The different balanced datasets where tested on several different binary classifiers, such as KNN, SVM, Bagging, and XGBooster. The highest accruacy we achieved is with XGBooster and Bagging with a 99% accuracy rate. The downside for this accuracy is overfitting as the minority class has a 0% predicted accuracy. The classifier which has the best accuracy equilibrium in proportion to the minority and majority class is XGBooster with Undersampling. This had a 60% overall accuracy rate, with 60% for injured prediction and 60% for non-injured prediction. Neither of these predicted accuracies are useful. The high accuracy overfits, and the lower accuracy is not good enough for real life application.